## Create Data

In [ ]:
# Sample sentences for the list
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "She opened her book and started to read.",
    "Today is a sunny day.",
    "The cat sat on the mat.",
    "He loves to play soccer.",
    "I need to buy some groceries.",
    "Python programming is fun.",
    "The lake is deep and beautiful.",
    "She sings beautifully.",
    "They are planning a trip to Paris."
]

# Print the list of sentences
for sentence in sentences:
    print(sentence)

## Installation

In [ ]:
! pip install openai

## Grab Secrets

In [ ]:
from google.colab import userdata

## Embeddings

In [ ]:
from openai import OpenAI
from typing import List

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def list_to_nums(sentences: List[str]) -> List[List[float]]:
    """
    Converts a list of sentences into a list of numerical embeddings using OpenAI's embedding model.

    Args:
    - sentences (List[str]): A list of sentences (strings).

    Returns:
    - List[List[float]]: A list of lists of numerical embeddings.
    """

    # Initialize the list to store embeddings
    embeddings = []

    # Loop through each sentence to convert to embeddings
    for sentence in sentences:
        # Use the OpenAI API to get embeddings for the sentence

        response = client.embeddings.create(
            input=sentence,
            model="text-embedding-3-small"
        )

        embeddings.append(response.data[0].embedding)

    return embeddings

In [ ]:
%%time

# Call the function
query_database = list_to_nums(sentences)

In [ ]:
prompt = "What is the meaning of life?"

In [ ]:
%%time

prompt_embed_ = list_to_nums([prompt])
print(type(prompt_embed_), len(prompt_embed_[0]))

## Quantized Influence Measure (QIM)

In [ ]:
import numpy as np
from typing import Any, Dict, List, Tuple, Union


def quantize_to_kbit(arr: Union[np.ndarray, Any], k: int = 16) -> np.ndarray:
    """Converts an array to a k-bit representation by normalizing and scaling its values.

    Args:
        arr (Union[np.ndarray, Any]): The input array to be quantized.
        k (int): The number of levels to quantize to. Defaults to 16 for 4-bit quantization.
    Returns:
        np.ndarray: The quantized array with values scaled to 0 to k-1.
    """
    if not isinstance(arr, np.ndarray):  # Check if input is not a numpy array
        arr = np.array(arr)  # Convert input to a numpy array
    arr_min = arr.min()  # Calculate the minimum value in the array
    arr_max = arr.max()  # Calculate the maximum value in the array
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)  # Normalize array values to [0, 1]
    return np.round(normalized_arr * (k - 1)).astype(int)  # Scale normalized values to 0-(k-1) and convert to integer


def quantized_influence(arr1: np.ndarray, arr2: np.ndarray, k: int = 16, use_dagger: bool = False) -> Tuple[float, List[float]]:
    """
    Calculates a weighted measure of influence based on quantized version of input arrays and optionally applies a transformation.

    Args:
        arr1 (np.ndarray): First input array to be quantized and analyzed.
        arr2 (np.ndarray): Second input array to be quantized and used for influence measurement.
        k (int): The quantization level, defaults to 16 for 4-bit quantization.
        use_dagger (bool): Flag to apply a transformation based on local averages, defaults to False.
    Returns:
        Tuple[float, List[float]]: A tuple containing the quantized influence measure and an optional list of transformed values based on local estimates.
    """
    # Quantize both arrays to k levels
    arr1_quantized = quantize_to_kbit(arr1, k)
    arr2_quantized = quantize_to_kbit(arr2, k)

    # Find unique quantized values in arr1
    unique_values = np.unique(arr1_quantized)

    # Compute the global average of quantized arr2
    total_samples = len(arr2_quantized)
    y_bar_global = np.mean(arr2_quantized)

    # Compute weighted local averages and normalize
    weighted_local_averages = [(np.mean(arr2_quantized[arr1_quantized == val]) - y_bar_global)**2 * len(arr2_quantized[arr1_quantized == val])**2 for val in unique_values]
    qim = np.sum(weighted_local_averages) / (total_samples * np.std(arr2_quantized))  # Calculate the quantized influence measure

    if use_dagger:
        # If use_dagger is True, compute local estimates and map them to unique quantized values
        local_estimates = [np.mean(arr2_quantized[arr1_quantized == val]) for val in unique_values]
        daggers = {unique_values[i]: v for i, v in enumerate(local_estimates)}  # Map unique values to local estimates

        def find_val_(i: int) -> float:
            """Helper function to map quantized values to their local estimates."""
            return daggers[i]

        # Apply transformation based on local estimates
        daggered_values = list(map(find_val_, arr1_quantized))
        return qim, daggered_values
    else:
        # If use_dagger is False, return the original quantized arr1 values
        daggered_values = arr1_quantized.tolist()
        return qim

In [ ]:
len(prompt_embed_[0]), len(query_database[0])

In [ ]:
%%time

current_qim = quantized_influence(prompt_embed_[0], query_database[0], k=16, use_dagger=False)
print(current_qim)

In [ ]:
%%time

current_qim = quantized_influence(prompt_embed_[0], query_database[0], k=3, use_dagger=False)
print(current_qim)

In [ ]:
len(query_database)

In [ ]:
%%time


scores = [[sentences[i], query_database[i], quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)] for i in range(len(query_database))]
print(scores)

In [ ]:
import pandas as pd

In [ ]:
%%time

refs = pd.DataFrame(scores)
refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "qim"})

In [ ]:
refs.sort_values(by="qim", ascending=False)

## Query Search

In [ ]:
import pandas as pd
from typing import List

def query_search(prompt: str) -> pd.DataFrame:
    """
    Takes a text prompt and searches a predefined database by converting the prompt
    and database entries to embeddings, and then calculating a quantized influence metric.

    Args:
    - prompt (str): A text prompt to search for in the database.

    Returns:
    - pd.DataFrame: A pandas DataFrame sorted by the quantized influence metric in descending order.
                     The DataFrame contains the original sentences, their embeddings, and the computed scores.
    """
    # Convert the prompt to its numerical embedding
    prompt_embed_ = list_to_nums([prompt])

    # Calculate scores for each item in the database using the quantized influence metric
    scores = [
        [
            sentences[i],  # The sentence itself
            query_database[i],  # Embedding of the sentence
            quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)  # Score calculation
        ]
        for i in range(len(query_database))
    ]

    # Convert the list of scores into a DataFrame
    refs = pd.DataFrame(scores)
    # Rename columns for clarity
    refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "qim"})
    # Sort the DataFrame based on the 'qim' score in descending order
    refs = refs.sort_values(by="qim", ascending=False)

    return refs

In [ ]:
query_search("What is the meaning of life?")

In [ ]:
query_search("The lake is dup and beauti")

In [ ]:
query_search("The planning to paris is not good")

## Read Data

In [ ]:
!pip install PyPDF2

In [ ]:
from typing import List, Tuple
import PyPDF2

def read_and_textify(files: List[str]) -> Tuple[List[str], List[str]]:
    """
    Reads PDF files from given paths and extracts text from each page.

    This function iterates over a list of PDF file paths, opens each file, extracts text from each page,
    and compiles a list of texts and corresponding source information.

    Args:
    files (List[str]): A list of paths to PDF files.

    Returns:
    Tuple[List[str], List[str]]: A tuple containing two lists:
        1. A list of strings, where each string is the text extracted from a PDF page.
        2. A list of strings indicating the source of each text (file name and page number).
    """

    text_list = []  # List to store extracted text
    sources_list = []  # List to store source information

    # Iterate over each file path
    for file_path in files:
        # Open the PDF file
        with open(file_path, "rb") as file:
            pdfReader = PyPDF2.PdfReader(file)  # Create a PDF reader object
            # Iterate over each page in the PDF
            for i in range(len(pdfReader.pages)):
                pageObj = pdfReader.pages[i]  # Get the page object
                text = pageObj.extract_text()  # Extract text from the page
                text_list.append(text)  # Add extracted text to the list
                # Extract the file name from the path and append the source info
                sources_list.append(f"{file_path.split('/')[-1]}_page_{i}")

    return text_list, sources_list


In [ ]:
%%time

uploaded_files = "/content/Understanding_and_Improving_Use_Tax_Compliance__A_Theory_of_Plann.pdf"

# Process the uploaded files to extract text and source information
textify_output = read_and_textify([uploaded_files])

# Separate the output into documents (text) and their corresponding sources
documents, sources = textify_output

In [ ]:
type(documents), type(sources)

In [ ]:
documents[0]

In [ ]:
len(documents), len(sources)

In [ ]:
documents[0], sources[0]

In [ ]:
%%time

# Call the function
query_database = list_to_nums(documents)

In [ ]:
import pandas as pd
from typing import List

def query_search(prompt: str, sentences: list[str], query_database: list[list[float]], sources: list[str]) -> pd.DataFrame:
    """
    Takes a text prompt and searches a predefined database by converting the prompt
    and database entries to embeddings, and then calculating a quantized influence metric.

    Args:
    - prompt (str): A text prompt to search for in the database.

    Returns:
    - pd.DataFrame: A pandas DataFrame sorted by the quantized influence metric in descending order.
                     The DataFrame contains the original sentences, their embeddings, and the computed scores.
    """
    # Convert the prompt to its numerical embedding
    prompt_embed_ = list_to_nums([prompt])

    # Calculate scores for each item in the database using the quantized influence metric
    scores = [
        [
            sentences[i],  # The sentence itself
            query_database[i],  # Embedding of the sentence
            sources[i],  # Source of the sentence
            quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)  # Score calculation
        ]
        for i in range(len(query_database))
    ]

    # Convert the list of scores into a DataFrame
    refs = pd.DataFrame(scores)
    # Rename columns for clarity
    refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "page no", 3: "qim"})
    # Sort the DataFrame based on the 'qim' score in descending order
    refs = refs.sort_values(by="qim", ascending=False)

    return refs

In [ ]:
%%time

ref_tab = query_search("pful for understanding federal income", documents, query_database, sources)

In [ ]:
ref_tab.to_json()